In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from geopy.distance import geodesic
import geopandas as gpd
import folium
from folium import Marker

In [2]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
df = pd.read_csv('data/반출용/FINAL_TEMP.csv')
df.head(2)

,RN,geometry,대형마트_개수,편의점_개수,지하철역_개수,은행_개수,문화시설_개수,공공기관_개수,카페_개수,병원_개수,약국_개수,헬스클럽_개수,셀프빨래방_개수,집객시설_총_개수,상주인구_수,도로_폭,도로_길이,집랭크,상랭크,가점수
0,강서로17길,POINT (126.845671 37.533482),0,7,1,5,0,0,15,15,6,9,0,58,14354,11,780,11,12,23
1,곰달래로19나길,POINT (126.844676 37.530839),1,6,0,2,0,1,15,15,2,6,2,50,15012,5,264,16,10,26


In [4]:
df.columns

Index(['RN', 'geometry', '대형마트_개수', '편의점_개수', '지하철역_개수', '은행_개수', '문화시설_개수',
       '공공기관_개수', '카페_개수', '병원_개수', '약국_개수', '헬스클럽_개수', '셀프빨래방_개수',
       '집객시설_총_개수', '상주인구_수', '도로_폭', '도로_길이', '집랭크', '상랭크', '가점수'],
      dtype='object')

In [5]:
# Folium 지도 객체 생성
m = folium.Map(location=[37.53, 126.84], zoom_start=15)  # 초기 지도 중심 및 확대 수준 설정

# 데이터를 지도에 추가
for geom in df['geometry']:
    coords = [float(coord) for coord in geom.strip('POINT ()').split()]
    marker = Marker(location=[coords[1], coords[0]])  # 위도와 경도를 역으로 넣음
    marker.add_to(m)


# 지도를 출력
m

In [6]:
df = df[['geometry', '가점수']].sort_values(by = '가점수', ascending = True)
df['Longitude'] = df['geometry'].str.extract(r'POINT \((\d+\.\d+) (\d+\.\d+)\)')[0].astype(float)
df['Latitude'] = df['geometry'].str.extract(r'POINT \((\d+\.\d+) (\d+\.\d+)\)')[1].astype(float)
df.head(3)

,geometry,가점수,Longitude,Latitude
0,POINT (126.845671 37.533482),23,126.845671,37.533482
1,POINT (126.844676 37.530839),26,126.844676,37.530839
2,POINT (126.845433 37.530496),36,126.845433,37.530496


In [7]:
result_points = []  # 최종 결과로 남길 점들을 담을 리스트

while not df.empty:
    # 인덱스 0번 점을 리스트에 추가
    result_points.append(df.iloc[0]['geometry'])
    
    # 인덱스 0번 점으로부터 00미터 이내의 점을 제거
    reference_point = df.iloc[0]['geometry']
    df = df[df.apply(lambda row: geodesic((df.iloc[0]['Latitude'], df.iloc[0]['Longitude']), (row['Latitude'], row['Longitude'])).meters > 300, axis=1)]
    # df = df[df.apply(lambda row: geodesic((df.iloc[0]['Latitude'], df.iloc[0]['Longitude']), (row['Latitude'], row['Longitude'])).meters > 590, axis=1)]

    # 데이터프레임이 비어있지 않은 경우, 인덱스를 다음으로 업데이트
    if not df.empty:
        df = df.reset_index(drop=True)

# 최종 결과 데이터프레임 생성
result_df = pd.DataFrame({'geometry': result_points})

In [8]:
result_df.shape

(11, 1)

In [9]:
m = folium.Map(location=[37.53, 126.84], zoom_start=15)  # 초기 지도 중심 및 확대 수준 설정

# 데이터를 지도에 추가
for geom in result_df['geometry']:
    coords = [float(coord) for coord in geom.strip('POINT ()').split()]
    marker = Marker(location=[coords[1], coords[0]])  # 위도와 경도를 역으로 넣음
    marker.add_to(m)


# 지도를 출력
m

In [10]:
result_df

,geometry
0,POINT (126.845671 37.533482)
1,POINT (126.844676 37.530839)
2,POINT (126.84081 37.529755)
3,POINT (126.838498 37.533759)
4,POINT (126.839934 37.539683)
5,POINT (126.847877 37.529743)
6,POINT (126.841965 37.534317)
7,POINT (126.835605 37.540346)
8,POINT (126.836279 37.537027)
9,POINT (126.843087 37.537657)


In [11]:
import folium

m = folium.Map(location=[37.53, 126.84], zoom_start=15)  # 초기 지도 중심 및 확대 수준 설정

# 데이터를 지도에 추가
for geom in result_df['geometry']:
    coords = [float(coord) for coord in geom.strip('POINT ()').split()]
    marker = folium.Marker(location=[coords[1], coords[0]])  # 위도와 경도를 역으로 넣음
    marker.add_to(m)

    # Add a circle with a radius of 300 meters
    circle = folium.Circle(
        location=[coords[1], coords[0]],  # Use the coordinates of the point
        radius=300,  # Radius in meters
        color='blue',  # Color of the circle outline
        fill=True,  # Fill the circle
        fill_color='blue'  # Fill color
    )
    circle.add_to(m)

# 지도를 출력
m


In [12]:
# another

In [13]:
df1 = pd.read_csv('data/반출용/FINAL_TEMP.csv')

In [14]:
df1 = df1[['geometry', '가점수']].sort_values(by = '가점수', ascending = True)
df1['Longitude'] = df1['geometry'].str.extract(r'POINT \((\d+\.\d+) (\d+\.\d+)\)')[0].astype(float)
df1['Latitude'] = df1['geometry'].str.extract(r'POINT \((\d+\.\d+) (\d+\.\d+)\)')[1].astype(float)
df1.head(3)

,geometry,가점수,Longitude,Latitude
0,POINT (126.845671 37.533482),23,126.845671,37.533482
1,POINT (126.844676 37.530839),26,126.844676,37.530839
2,POINT (126.845433 37.530496),36,126.845433,37.530496


In [15]:
result_points2 = []  # 최종 결과로 남길 점들을 담을 리스트

while not df1.empty:
    # 인덱스 0번 점을 리스트에 추가
    result_points2.append(df1.iloc[0]['geometry'])
    
    # 인덱스 0번 점으로부터 00미터 이내의 점을 제거
    reference_point2 = df1.iloc[0]['geometry']
    df1 = df1[df1.apply(lambda row: geodesic((df1.iloc[0]['Latitude'], df1.iloc[0]['Longitude']), (row['Latitude'], row['Longitude'])).meters > 580, axis=1)]

    # 데이터프레임이 비어있지 않은 경우, 인덱스를 다음으로 업데이트
    if not df1.empty:
        df1 = df1.reset_index(drop=True)

# 최종 결과 데이터프레임 생성
result_df2 = pd.DataFrame({'geometry': result_points2})

In [16]:
m2 = folium.Map(location=[37.53, 126.84], zoom_start=15)  # 초기 지도 중심 및 확대 수준 설정

# 데이터를 지도에 추가
for geom in result_df2['geometry']:
    coords = [float(coord) for coord in geom.strip('POINT ()').split()]
    marker2 = Marker(location=[coords[1], coords[0]])  # 위도와 경도를 역으로 넣음
    marker2.add_to(m2)


# 지도를 출력
m2

In [17]:
import folium

m2 = folium.Map(location=[37.53, 126.84], zoom_start=15)  # 초기 지도 중심 및 확대 수준 설정

# 데이터를 지도에 추가
for geom in result_df2['geometry']:
    coords = [float(coord) for coord in geom.strip('POINT ()').split()]
    marker2 = folium.Marker(location=[coords[1], coords[0]])  # 위도와 경도를 역으로 넣음
    marker2.add_to(m2)

    # Add a circle with a radius of 300 meters
    circle = folium.Circle(
        location=[coords[1], coords[0]],  # Use the coordinates of the point
        radius=300,  # Radius in meters
        color='blue',  # Color of the circle outline
        fill=True,  # Fill the circle
        fill_color='blue'  # Fill color
    )
    circle.add_to(m2)

# 지도를 출력
m2
